# Motion control

## Connection/ disconnection functions

Initially, functions facilitating the establishment and termination of the connection with the robot have been implemented.

In [1]:
# Thymio connection
async def connect_Thymio():
    """
    Establish a connection with the Thymio if possible
    """
    global node, client
    try:
        client = ClientAsync()
        node = await asyncio.wait_for(client.wait_for_node(), timeout=2.0)
        await node.lock()
        print("Thymio connected")

    except asyncio.TimeoutError:
        print("Thymio not connected: Timeout while waiting for node.")
    except Exception as e:
        print(f"Thymio not connected: {str(e)}")
        
# Thymio disconnection
def disconnect_Thymio():
    """
    Enable to disconnect the Thymio
    """
    aw(node.stop())
    aw(node.unlock())
    print("Thymio disconnected")

## Motion functions

The robot’s movement from its starting position to the target is executed using the coordinates of the global path. To transition from one coordinate to the next, a specific method has been adopted. The robot begins by orienting itself towards the next point, taking into account its current orientation. Once the orientation is adjusted, the robot performs a straight-line movement to the next point. This step is repeated from position to position until the final destination is reached. The basic functions therefore include: activation and deactivation of motors, rotation by a defined angle, and linear movement over a defined distance.

In [3]:
# Thymio set motors speeds  
async def set_speeds(left_speed, right_speed):
    """
    Enable to set the speed of the Thymio's wheels
    """
    global node
    v = {
        "motor.left.target":  [left_speed],
        "motor.right.target": [right_speed],
    }
    await node.set_variables(v)
    
# Thymio motors stop     
async def motors_stop():
    """
    Stop the Thymio
    """
    global node
    v = {
        "motor.left.target":  [0],
        "motor.right.target": [0],
    }
    await node.set_variables(v)    

# Conversion factors
Thymio_to_mms = 0.349
px_to_mm = 137/100
# Constants
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)

# Thymio turns a specied angle
async def turn(angle):
    # Calculate the time needed to turn through the required angle
    rotation_time = (abs(angle) / (2*np.pi)) * TIME_FULL_TURN

    # Turn robot on itself
    # Check the sign of angle
    if np.sign(angle) > 0:
        # If angle is positive, turn left
        await set_speeds(-ROTATION_SPEED, ROTATION_SPEED)
        left_or_right = TURN_LEFT
    else:
        # If angle is negative, turn right
        await set_speeds(ROTATION_SPEED, -ROTATION_SPEED)
        left_or_right = TURN_RIGHT

    # Wait required time
    time.sleep(rotation_time)
    return rotation_time, left_or_right

# Constants
FORWARD_SPEED = 200 
TIME_PER_MM = 15.5/1000  # Time it takes for the robot to travel one meter at base speed

async def move_forward(distance_px):
    # Calculate the time needed to travel the requested distance
    
    distance_mm = distance_px * px_to_mm
    travel_time = (distance_mm) * TIME_PER_MM
    
    # Robot moves forward
    await set_speeds(FORWARD_SPEED, FORWARD_SPEED)

    # Wait for the necessary time
    time.sleep(travel_time)
    return travel_time

## Motion control function

The management of one coordinate to another is entirely handled by the following function, which determines whether the next movement is a rotation or a translation, and establishes the angle or distance to be covered.

In [4]:
async def reach_next_node(next_node, mode, estimated_pos):

    # Vector between the estimated position and the next position in the global path
    vector_next_node = np.array([0,0])  
    vector_next_node[0] = path_final[0][next_node] - estimated_pos[0]
    vector_next_node[1] = path_final[1][next_node] - estimated_pos[1] 
    
    # Normalized angle between the estimated position and the next position in the global path
    gamma = -math.atan2(vector_next_node[1], vector_next_node[0]) - estimated_pos[2]
    gamma = (gamma + np.pi) % (2*np.pi) - np.pi
    
    # Distance separating the estimated position and the next position in the global path
    path_next_node = np.array([path_final[0][next_node], path_final[1][next_node]])
    path_current_node = np.array([estimated_pos[0], estimated_pos[1]])
    d = np.linalg.norm(path_next_node - path_current_node)
    
    if(not mode):
        if(abs(gamma) > ANGLE_THRESHOLD):
            time_r, left_or_right = await turn(gamma)
        else: 
            time_r = 0 
            left_or_right = 1  
        return time_r, left_or_right 
        
    if (mode):
        if( d > FORWARD_THRESHOLD):
            time_f = await move_forward(d)
            return time_f